# Simulación para los smart contracts


In [ ]:
# Importación de librerias 

# Númericas
import numpy as np         # Para trabajar arreglos matriciales
import random              # Para determinar valores aleatorios
import scipy.stats as ss   # Aleatoriedad atada a una distribución de probabilidad
import math 

# Gráficas
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize
import plotly.graph_objects as go

## Parámetros del protocolo

Para definir los mercados que harán parte del protocolo, se deben definir los parámetros que definirán comportamientos especificos en el modelo de *lending and borrow*, tales como el porcentaje de reservas requerido, la tasa de garantía o colateral, etc.

*  **Collateral Factor:** Es el porcentaje que se puede tomar prestado de un activo en base a lo que se suministra. Por ejemplo, supongamos que suministramos $100\text{ TK1}$ (tokens 1) al protocolo, actualmente el mercado $\text{TK1}$ cuenta con un collateral factor del $75\%$, esto nos indica que podemos tomar prestado hasta $75\text{ TK1}=100\text{ TK1}\cdot75\%$.

*  **Reserve Factor:** Es el porcentaje de la cantidad suministrada que se va a "guardar" como reservas en el protocolo.

*  **Modelo de tasas de interés:** Es la expresión matemática encargada de asignar una tasa de interés a cada mercado en función de la oferta y la demanda presentes en cada mercado.

$$\text{Borrow APR}=0.02+0.1\cdot\text{utilization rate}$$

*  **Exchange rate initial:** Es la tasa de cambio inicial entre los tokens y los cTokens. Por ejemplo, si consideramos el mercado $\text{TK1}$ y sabemos que el equivalente a $1\text{ TK1}$ en cTK1 es $0.02\text{ cTK1}$, el exchange rate actual es $0.02\text{ cTK1}$.

A continuación definiremos los parámetros de dos mercados, $\text{MK1}$ y $\text{MK2}$:

In [ ]:
# Numero de bloques por año en la red

ETHBlocks = 4*60*24*365
RSKBlocks = ETHBlocks/2

# Collateral Factors - Uno por cada mercado

collFactor_MK1 = 0.75  # Collateral factor para el mercado MK1
collFactor_MK2 = 0.75  # Collateral factor para el mercado MK2

# Lista con los Collateral Factors
collateralFactors = [collFactor_MK1,collFactor_MK2]

# Reserve Factors - Uno por cada mercado

reserveFactor_MK1 = 0.20   # Reserve Factor para el mercado MK1
reserveFactor_MK2 = 0.15   # Reserve Factor para el mercado MK2
 
# Lista con los Reserve Factors                    
reserveFactors = [reserveFactor_MK1,reserveFactor_MK2]

# Mínimo suministrado permitido

minAmount = 1*10**(-18)

# Máximo permitido por mercado

maxAmountMK1 = 1  # 1 BTC
maxAmountMK2 = 10000  # 10000 USDC
maxAmount = [maxAmountMK1,maxAmountMK2]

# Precios iniciales por mercado

priceMK1 = 50000  # BTC price in USD
priceMK2 = 1.01  # USDC price in USD
initialMarketPrices = [priceMK1,priceMK2]

In [ ]:
# El modelo de tasas de interés - Una expresión para todos los mercados

def getBorrowAPR_volatileMarket(utilizationRate,baseBorrowRate=0.02,slope=0.1):   
    # Borrow APR MK1
    return baseBorrowRate + slope*utilizationRate

def getBorrowAPR_stableMartet(utilizationRate,baseBorrowRate=0.02,slope1=0.05,slope2=1,optimalUtilization=0.8):   
    # Borrow APR MK2
    if utilizationRate <= optimalUtilization:
        return baseBorrowRate+slope1*utilizationRate
    else:
        return (baseBorrowRate+(slope1-slope2)*optimalUtilization) + slope2*utilizationRate

def getSupplyAPR(borrowRate,reserveFactor,utilizationRate):  
    # Supply APR
    return borrowRate*utilizationRate*(1-reserveFactor)

def ratePerBlock(rateAPR,blockPerYear=ETHBlocks):
    # Rate por bloque
    rate = rateAPR/blockPerYear
    return rate 

## Estructura de los mercados

Dado que el estado de cada mercado se actualiza en cada bloque, debemos implementar una manera adecuada de conocer el estado del mercado y además, el estado de cada usuario (balance de la cuenta y movimientos de cada usuario). 

![texto del enlace](Imagenes/market_structure.jpg)

Cada mercado en cada bloque tiene un estado similar a la imágen anterior, cada casilla cumple una función especifica:
1.   **Borrow rate:** Es la tasa de prestamo actual, esta se calcula en base a la tasa de utilización del bloque anterior.
2.   **Supply rate:** Es la tasa de suministro actual, o de otra forma, será la tasa a la cual el lender esta ganando intereses en función de su capital suministrado. Al igual que el *borrow rate*, se calcula en base al bloque anterior.
3.   **Token price:** Es el precio actual del mercado, es una variable independiente del protocolo y viene dada por un oraculo.
4.  **Total pool reserves:** Es la cantidad de reservas actuales en el mercado.
5.  **Exchange rate:** Tasa de cambio entre tokens y ctokens en el bloque actual.
6.  **t:** Es la cantidad de bloques que lleva la tasa *borrow rate*, tambien se puede entender como la cantidad de bloques en las que no se ha hecho ninguna acción por parte de los usuarios.
7.  **Total borrow:** Es la cantidad total de tokens prestados (junto con intereses) en el mercado en el bloque actual.
8.  **Total supply:** Es la cantidad total de tokens suministrados en el mercado.
9.  **Movimiento de usuarios y registro de acciones:** En esta casilla se guardan las carteras y las acciones que realizará cada usuario en el bloque actual, cada elemento en esta casilla se puede ver como: 
![texto del enlace](Imagenes/user_structure.jpg)

Todos los usuarios que hagan parte del mercado poseen las caracteristicas o datos que se ven en el diagrama anterior: 
*   En la primera casilla encontramos la *acción* que realiza el usuario, en cada bloque cada usuario puede realizar una de cinco acciones: *supply* (o suministrar liquidez), *borrow*, *withdraw*, *repay* o no hacer nada.
*   En la segunda casilla encontramos el amount, será el monto o la cantidad de tokens que se emplean dependiendo de la acción.
*   En la tercera y cuarta posición encontramos los datos correspondientes al supply del usuario, en la tercera los montos que suministra el usuario y en la cuarta lo suministrado mas los intereses que se van acumulando.
*   Finalmente en las quinta y sexta posición encontramos los datos correspondientes a los prestamos que realice el usuario en el mercado. De manera análoga se guardan en la quinta posición los montos que se solicitan como prestamo y en la sexta posición los montos prestados mas los intereses que se generan por la deuda.


## Configuración de un mercado

Para crear el entorno inicial debemos tener presente varios factores que influiran de manera significativa en nuestra simulación.

Lo primero es establecer la cantidad de usuarios que vamos a considerar, está debe ser la misma para todos los mercados que se quieran crear. Una vez establecido este valor debemos "llamar" los valores ya establecidos por gobernanza y podremos crear tantos mercados como necesitemos.

In [ ]:
# Formato de visualización del mercado

def fixLen(value, isNumber=False):
    if isNumber: 
        value = '%.5f' % value
        value = value[:9]
    return '{:9s}'.format(str(value))

def printMarket(market):
    blockNumber = 0
    for marketAtblock in market:
        #print('InterestIndexArray:', marketAtblock[9])
        print('accrualBlockNumber:', marketAtblock[5], '\t exchangeRate:', marketAtblock[4],
              '\t marketPrice:', marketAtblock[2])
        print('totalSupplies:', marketAtblock[7], '\t totalBorrows:', marketAtblock[6],
              '\t totalReserves:', marketAtblock[3])
        print('borrowRateAPR:', marketAtblock[0], '\t supplyRateAPR:', marketAtblock[1],
              '\t MarketCash:', marketAtblock[8])
        print('MarketIndex:', marketAtblock[9][-1],'\n')
        print('|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|')
        print('| ',fixLen('# Block'),' | ',fixLen('UserIdx'),' | ',fixLen('Action'),' | ',
              fixLen('Amount'),' | ',fixLen('Supply'),' | ',fixLen('SPlusI'),' | ',
              fixLen('Borrow'),' | ',fixLen('IUIdx'),' |')    
        print('|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|')
        userIndex = 0
        for user in marketAtblock[10:]:
            print('| ',fixLen(blockNumber),' | ',fixLen(userIndex),' | ',fixLen(user[0]),' | ',fixLen(user[1], True),' | ',fixLen(user[2], True),' | ',fixLen(user[3], True),' | ',fixLen(user[4], True),' | ',fixLen(user[5]),' |')
            userIndex += 1
        blockNumber+=1
        print('|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|')

In [ ]:
def marketSetUp(n_users,reserveFactor,initialSuppliersRate,initialTokenPrice,maxAmountSupply,interestModel):
    market = []  
    borrowRate = interestModel(0)
    supplyRate = getSupplyAPR(borrowRate,reserveFactor,0)
    initialExchangeRate = 50
    market.append(borrowRate)   # Borrow rate with utilizationRate = 0, 0
    market.append(supplyRate)   # Supply rate with utilizationRate = 0, 1
    market.append(initialTokenPrice)  # Initial Price Market, 2
    market.append(0)  # Total Reserve, 3
    market.append(initialExchangeRate) # Exchange Rate, 1 token == 50 ctokens, 4
    market.append(0)  # Accrual block number, 5
    market.append(0)  # TotalBorrow in Market, 6
    market.append(0)  # TotalSupplied in Market, 7
    market.append(0)  # Market cash, 8
    market.append([ratePerBlock(borrowRate)])  # Interest index, 9
    for i in range(n_users):
        r = random.random()
        if r <= initialSuppliersRate:
            supplied = random.uniform(minAmount,maxAmountSupply)
            userSupply = ("s",supplied,supplied,supplied*initialExchangeRate,0,0)
            market.append(userSupply)
        else:
            market.append(("x",0,0,0,0,0))
    supply = 0
    for user in market[10:]:
        supply += user[2]
    reserve = supply*reserveFactor
    market[3] = reserve  # Market reserve
    market[7] = supply   # Market total supply
    market[8] = supply + reserve # Market Cash
    return [market]

M1 = marketSetUp(5,0.2,0.5,initialMarketPrices[0],maxAmount[0],getBorrowAPR_volatileMarket)
printMarket(M1)
M2 = marketSetUp(5,0.2,0.5,initialMarketPrices[1],maxAmount[1],getBorrowAPR_stableMartet)
printMarket(M2)

## Actualización de dinámicas del mercado

Dado que en cada bloque debemos conocer el estado del mercado, debemos establecer mecanismos para actualizar todas las posibles variables de cada mercado.

La manera en la que se actualizarán las tasas de interés vendrá dada por el modelo de interés y el comportamiento del mercado en el bloque anterior, esto quiere decir que en el bloque $n$ las tasas $\text{Borrow rate}$ y $\text{Supply rate}$ dependerán de la tasa de utilización (relación entre oferta y demanda) del bloque $n-1$.

Recordemos que la tasa de utilización esta dada por la siguiente ecuación:

$$\text{Utilization rate}=\frac{Total Borrow}{Total supply}\approx\frac{Total Borrow}{Total Supply+TotalBorrow}$$

De ese modo, construiremos las funciones que nos permitiran determinar los posibles cambios en las diferentes tasas del mercado en el último bloque generado:

# Stable coins
$$\text{Utilization rate}=\frac{Total Borrow}{Total Liquility + Total Borrow}$$

Borrow rate:

$$\text{Borrow rate}=\left\{\begin{array}{ll}
\text{baseBorrowRate}+\text{slope}_{1}*\text{UtilizationRate} & \text{si } utilizationRate\leq kink\\
\text{crossPoint}+\text{solpe}_{2}*\text{UtilizationRate} & \text{otherwise}\end{array}\right.$$

Supply rate:

$$\text{Supply rate}={utilization Rate*borrowRate}*({1-reserveFator})$$

In [ ]:
testMarket = [[0.02,0,10,2,50,0,0,10,8,[0.02/(4*60*24*365)],("s",10,10,10*50,0,0)],
              [0.02,0,10,2,50,0,5,10,8,[0.02/(4*60*24*365),(0.02/(4*60*24*365))*(1+(0.02/(4*60*24*365)))],("b",5,10,10*50,5,0)]]
printMarket(testMarket)

In [ ]:
# Calculo de la tasa de utilización del mercado en el bloque actual

def getUtilizationRate(market,blockNumber,reserveFactor):  # Recibe el nuevo bloque
    if blockNumber == 1 or blockNumber == 0:
        return 0
    # Utilización con la que empieza el nuevo bloque
    borrowRate = market[blockNumber-1][0]
    totalBorrow = market[blockNumber-1][6]  # Lo que habia prestado mas lo que paso en el bloque anterior
    for user in market[blockNumber-1][10:]:
        totalBorrow += user[4]
    cash = market[blockNumber-1][8]
    return totalBorrow/(cash+totalBorrow)  # Retorna un valor porcentual

# Calculo de la tasa Borrow rate (por el momento se esta tomando la tasa APR)

def borrowRateAPR(market,blockNumber,interestModel,reserveFactor): # Recibe el nuevo bloque
    # Calcula la tasa de interés que se debe pagar por los borrowers por bloque
    utilizationRate = getUtilizationRate(market, blockNumber, reserveFactor)
    borrowAPR = interestModel(utilizationRate)
    return borrowAPR  # Retorna un valor porcentual

# Calculo de la tasa Supply rate (por el momento se esta tomando la tasa APR)

def supplyRateAPR(market,blockNumber,reserveFactor,borrowRatePerBlock):  # Recibe el nuevo bloque
    # Calcula la tasa de interés que se paga a los suppliers por bloque
    utilizationRate = getUtilizationRate(market, blockNumber, reserveFactor)
    supplyAPR = getSupplyAPR(borrowRatePerBlock, reserveFactor, utilizationRate)
    return supplyAPR  # Retorna un valor porcentual

UR_M1 = getUtilizationRate(M1,0,0.2)
BOR_M1 = borrowRateAPR(M1,0,getBorrowAPR_volatileMarket, 0.2)
SUP_M1 = supplyRateAPR(M1,0,reserveFactors[0],BOR_M1)
print("La tasa de utilización del mercado MK1 en el bloque cero es %.4f" %UR_M1)
print("El Borrow rate para el bloque 1 en el mercado MK1 será %.4f" %BOR_M1)
print("El Supply rate para el bloque 1 en el mercado MK1 será %.4f" %SUP_M1)

In [ ]:
def hasChangeBorrowRate(market,blockNumber,newBorrowRate):  # Se recibe el nuevo blockNumber 
  # Se va a verificar si cambia el valor para actualizar el delta y el interestIndex para el bloque blockNumber
  return market[blockNumber-1][0] != newBorrowRate  # Si es True, la tasa cambio

# Funcion para calcular los delta, i.e. espacio entre acciones en numero de bloques

def accrualBlockNumber(market, blockNumber, newBorrowRate):  # Se recibe el nuevo blockNumber
    # Reporta el valor de delta y el ultimo indice (rate por bloque) calculado
    if blockNumber == 0:
        return (0,[ratePerBlock(newBorrowRate)])
    borrowRateChanged = hasChangeBorrowRate(market,blockNumber,newBorrowRate)  # New Borrow Rate
    if borrowRateChanged == False:  # No cambio el borrow rate
        indexes = market[blockNumber-1][9]
        delta = market[blockNumber-1][5]+1
        return (delta, indexes)
    else:  # Si la tasa cambio
        delta = market[blockNumber-1][5]+1
        # I_n = I_n-1 (1 + borrowRatePerBlock_n-1 * delta)
        lastI = market[blockNumber-1][9][-1]
        indexPerBlock = lastI * (1 + ratePerBlock(newBorrowRate)*delta)
        indexes = market[blockNumber-1][9]
        temporalIndexes = []
        for i in indexes:
            temporalIndexes.append(i)
        temporalIndexes.append(indexPerBlock)
        return (0,temporalIndexes)

In [ ]:
# Saldo total suministrado en el mercado en tokens

def totalSupplyMarket(market,blockNumber):  # Se recibe el nuevo blockNumber
    supply = 0
    for user in market[blockNumber-1][10:]:
        supply += user[2]   # Se toman unicamente las cantidades suministradas sin intereses
    return supply

# Saldo total prestado + intereses del mercado en tokens

def totalBorrowMarket(market,blockNumber,newBorrowRate):  # Se recibe el nuevo blockNumber
    # Calcular la cantidad prestada mas intereses con las que inicia el nuevo bloque
    if blockNumber == 0 or blockNumber == 1:
        return 0
    delta, indexes = accrualBlockNumber(market,blockNumber,newBorrowRate)
    if delta != 0:
        return market[blockNumber-1][6]
    else:
        totalBorrow = 0   # Se consideran las acciones del bloque anterior
        totalRepay = 0
        for user in market[blockNumber-1][10:]:
            if user[0] == "b":
                totalBorrow += user[1]
            elif user[0] == "r":
                totalRepay += user[1]
        return (totalBorrow + market[blockNumber-1][6] - totalRepay)*(indexes[-1]/indexes[-2])  # En tokens
    
# Se actualizan las reservas de acuerdo con los withdraws del bloque anterior

def totalPreviousReserves(market,blockNumber):  # Se recibe el nuevo blockNumber
    withdraws = 0
    repays = 0
    for user in market[blockNumber-1][10:]:
        if user[0] == "w":
            withdraws += user[1]
        if user[0] == "r":
            repays += user[1]
    balanceMarket = market[blockNumber-1][8] - withdraws + repays  # Lo que habia - lo que se saco + las deudas que se pagaron
    if balanceMarket <= 0:
        newTotalReserve = market[blockNumber-1][3] + balanceMarket
    else:
        newTotalReserve = market[blockNumber-1][3]
    return newTotalReserve

def totalReserveMarket(market,blockNumber,reserveFactor,newBorrowRate):  # Se recibe el nuevo blockNumber
    # Calcular la cantidad de resrvas mas intereses con las que inicia el nuevo bloque
    previousTotalReserve = totalPreviousReserves(market,blockNumber)
    if blockNumber == 0:
        return market[0][3]
    totalBorrow = totalBorrowMarket(market,blockNumber,newBorrowRate)
    totalReserve = previousTotalReserve + totalBorrow*reserveFactor
    return totalReserve

# Liquidez total del mercado 

def getMarketCash(market,blockNumber,reserveFactor,newBorrowRate): # Recibe el nuevo bloque
    # Liquidez con la que empieza el nuevo bloque
    if blockNumber == 0:
        return market[0][7]-market[0][6]+market[0][3]
    supply = totalSupplyMarket(market,blockNumber) 
    borrow = totalBorrowMarket(market,blockNumber,newBorrowRate)  
    reserve = totalReserveMarket(market,blockNumber,reserveFactor,newBorrowRate) 
    cash = supply - borrow + reserve  # Identidad para el cash 
    return cash

In [ ]:
# Tasa de cambio para un mercado

def exchangeRateMarket(market,blockNumber,reserveFactor,totalSupply,totalBorrow,totalReserves,newBorrowRate):  # Recibe el nuevo bloque
    if blockNumber == 0:
        return 50   # exchangeRate inicial
    else:
        cash = getMarketCash(market,blockNumber,reserveFactor,newBorrowRate)
        totalBorrow = totalBorrowMarket(market,blockNumber,newBorrowRate)
        totalReserves = totalReserveMarket(market,blockNumber,reserveFactor,newBorrowRate)
        cTokenMinted = 0
        for user in market[blockNumber-1][10:]:
            cTokenMinted += user[3]
        if cash+totalBorrow-totalReserves != 0:
            exchangeRate = cTokenMinted/(cash+totalBorrow-totalReserves)
        else:
            exchangeRate = 50
        return exchangeRate

In [ ]:
def setMarketVariablesInBlock(market, blockNumber, reserveFactor, marketPrice, interestModel): # Recibe el nuevo bloque
    # Condiciones de mercado
    # --------------------------------------------------------------------------------------------------------------------
    borrowRate = borrowRateAPR(market,blockNumber,interestModel,reserveFactor)
    supplyRate = supplyRateAPR(market,blockNumber,reserveFactor,borrowRate)
    delta, indexes = accrualBlockNumber(market,blockNumber,borrowRate)
    totalSupply = totalSupplyMarket(market,blockNumber)
    totalBorrow = totalBorrowMarket(market,blockNumber,borrowRate)
    totalReserve = totalReserveMarket(market,blockNumber,reserveFactor,borrowRate)
    marketCash = getMarketCash(market,blockNumber,reserveFactor,borrowRate)
    exchangeRate = exchangeRateMarket(market,blockNumber,reserveFactor,totalSupply,totalBorrow,totalReserve,borrowRate)
    # --------------------------------------------------------------------------------------------------------------------
    mkAtBlock = [('x', 0, 0, 0, 0, 0)] * len(market[0])
    mkAtBlock[0] = borrowRate
    mkAtBlock[1] = supplyRate
    mkAtBlock[5] = delta 
    mkAtBlock[9] = indexes
    mkAtBlock[7] = totalSupply
    mkAtBlock[6] = totalBorrow
    mkAtBlock[3] = totalReserve
    mkAtBlock[8] = marketCash
    mkAtBlock[4] = exchangeRate
    mkAtBlock[2] = marketPrice
    # --------------------------------------------------------------------------------------------------------------------
    userIndex = 10
    for users in market[blockNumber-1][10:]: 
        mkAtBlock[userIndex] = market[blockNumber-1][userIndex] 
        userIndex += 1 
    return mkAtBlock

printMarket([setMarketVariablesInBlock(M1,1,0.2,50000,getBorrowAPR_volatileMarket)])

In [ ]:
printMarket(M1)

Para actualizar las demas variables que definen el estado del mercado en el nuevo bloque, debemos contar la cantidad de bloques en las que las tasas de interés no han sufrido cambios, es decir, bloques en los que no hubo cambios entre la oferta y la demanda del mercado. Para esto definiremos la función que cambiará el valor de $t$, el cual como ya mencionamos, será la cantidad de bloques sin cambios en las tasas de interés:

En base al valor de $t$ se calcularán de una u otra forma los intereses tanto del mercado como de los usuarios. Analicemos esto con un ejemplo:

Supongamos que estamos interesados en realizar un suministro al mercado MK1 y que al momento de realizar el suministro las tasas de interes son $\text{Borrow rate}=0.05$ y $\text{Supply rate}=0.02$; nuestros fondos creceran con la misma tasa de interés en tanto que ningún usuario interactue con el mercado MK1, es decir, la tasa de interés será la misma mientras que ningún otro usuario (o nosotros) realicemos un supply, un borrow, un withdraw o un repay.

En este cambio el interés será del tipo **simple**, con lo cual si suponemos que en 100 bloques no se ha realizado niguna acción, tendremos

$$\text{Capital}_{100} = \text{Capital}_0*(1+100*\text{Supply rate})$$

o de manera general,

$$\text{Capital}_{\text{bloque_n}} = \text{Capital}_{\text{bloque_0}}*(1+t*\text{Supply rate})$$

si en $n$ bloques no se ha realizado ninguna acción. Por otro lado, si el valor de $t$ es cero (esto implica que algún usuario realizó una acción en el mercado MK1), se deben capitalizar los intereses, en este punto el interés se transforma en un interés del tipo compuesto, con lo cual si suponemos que en el bloque 101 alguien realizo alguna acción, tendremos

\begin{align}
\text{Capital}_{101}&=\text{Capital}_{100}*(1+\text{New supply rate})\\
&=\text{Capital}_{0}*(1+100*\text{Supply rate})*(1+\text{New supply rate})
\end{align}

Realizando así, el proceso de capitalización de intereses en el momento que se actualizan las tasas del mercado.

![texto del enlace](Imagenes/interest.jpg)

De ese modo, debemos actualziar el saldo total prestado del mercado en base a los intereses (simples o compuesto) que se calculen.

Continuando con la actualización de los valores propios del mercado, tenemos el total de reservas del mercado. Al igual que el saldo total prestado, las reservas tambien se capitalzian de acuerdo a la actualización de las tasas del mercado, con lo cual, la manera en la que se calcularán las reservas en un bloque $n$ será:

$$\text{TotalReserves}_n=\left\{\begin{array}{ll}
\text{TotalReserves}_{0}+\text{TotalBorrow}_{0}*(\text{BorrowRate}*\Delta*\text{ReserveFactor}) & \text{si }\Delta\neq0\\
\text{TotalReserves}_{n-1}+\text{TotalBorrow}_{n-1}*(\text{BorrowRate}*\text{ReserveFactor}) & \text{si }\Delta=0\end{array}\right.$$

La expresión anterior se puede entender como que una parte (tanta como el factor de reserva) de los intereses generados por los prestamos, se irá para la reserva del mercado.

Es momento de establecer el mecanismo por el cual se define la tasa de cambio entre tokens y ctokens. Para comprender esta noción debemos tener presente los siguientes conceptos:

*  **Acuñar tokens (mint tokens):** Es el proceso de suministrar algún activo en el mercado y recibir ctokens a cambio.
*  **Saldo total de subyacente:** Es la cantidad de tokens que hay en el mercado.
*  **Saldo total prestado:** Es la cantidad de tokens prestados.
*  **Reservas:** Reservas actuales del mercado.

Con lo cual, la expresión que define la tasa de cambio en el bloque actual es

$$TokenPrice = \frac{1}{cTokenPrice}=\frac{TotalUnderlyingBalance+TotalBorrowBalance-Reserves}{cTokenMinted}$$

Una característica que cabe mencionar es que a pesar de que la tasa de cambio depende del saldo total suministrado y si este baja, la tasa de cambio tambien podrá bajar, el crecimiento en el valor de los cTokens se verá directamente relacionado con la acumulación de intereses generados por los prestamos de cada usuario.

Para determinar la cantidad $cTokenMinted$ debemos tomar la cantidad de tokens en el mercado (incluyendo los tokens que fueron prestados fuera de intereses) y dividirlos por el precio anterior de los cTokens.

Una vez obtenidos todos los valores anteriormente mencionados, podemos definir el estado del mercado en el siguiente bloque; es importante mencionar que en este punto aun no hemos tomado en cuenta los movimientos de los usuarios.

## Update de los movimientos de los usuarios

Para simular cada una de las acciones que pueden realizar los usuarios será necesario implementar una distribución de probabilidad que modele la cantidad promedio de ocurrencias de algún suceso en un tiempo especifico, en nuestro contexto se traduce por ejemplo, a la cantidad promedio de usuarios que hacen supply en un bloque.

La distribución de probabilidad la podemos entender como una expresión matemática que asigna a cada valor, la probabilidad de que "aparezca" dicho valor. 

En ese sentido, la distribución de probabilidad que mejor se ajusta a nuestra simulación es la distribución de Pöisson, la cual nos indicara la probabilidad de que uno o dos, o 100 usuarios realicen supply (por ejemplo) en un bloque dado.

Para comprender mejor la noción de distribución de probabilidad que vamos a implementar en la simulación realizaremos un ejemplo gráfico.

Supongamos que las condiciones del mercado son tales que el promedio de entradas de suppliers por bloque es 5, la cantidad de borrowers promedio es de 3, la de usuarios que realizan withdraw es de 1 y los que hacen un repay son 2 en promedio. Las distribuciones de probabilidad que se ajustan a estos valores son:


In [ ]:
# Valores estimados de ocerrencias

lambda_suppliers = 5    # Candidad estimada de llegada de suppliers por bloque  
lambda_borrowers = 3    # Candidad estimada de llegada de borrowes por bloque
lambda_withdraws = 1    # Candidad estimada de usuarios que realizan withdraw por bloque
lambda_repays = 2       # Candidad estimada de usuarios que realizan repay por bloque

# Distribuciones de Poisson para los valores dados

suppliersDistribuion = ss.poisson(lambda_suppliers)   
borrowersDistribuion = ss.poisson(lambda_borrowers)   
withdrawsDistribuion = ss.poisson(lambda_withdraws)   
repaysDistribuion = ss.poisson(lambda_repays)

# Visualización

people = np.arange(15)

suppliersProbability = []  
borrowersProbability = []
withdrawsProbability = []
repaysProbability = []

for user in people:
    suppliersProbability.append(suppliersDistribuion.pmf(user))   
    borrowersProbability.append(borrowersDistribuion.pmf(user))   
    withdrawsProbability.append(withdrawsDistribuion.pmf(user))
    repaysProbability.append(repaysDistribuion.pmf(user))

# Visualización
fig = go.Figure()

fig.add_trace(go.Scatter(x=people,y=suppliersProbability,name="Suppliers"))
fig.add_trace(go.Scatter(x=people,y=borrowersProbability,name="Borrowers"))
fig.add_trace(go.Scatter(x=people,y=withdrawsProbability,name="Withdraws"))
fig.add_trace(go.Scatter(x=people,y=repaysProbability,name="Repays"))

fig.update_layout(title="Distribuciones para la acción por parte de cada usuario por bloque",
                  xaxis_title='Cantidad de personas',
                  yaxis_title='Probabilidad')
fig.show()

En la gráfica anterior podemos ver que la probabilidad de que 4 usuarios hagan supply es mayor por ejemplo a la probabilidad de que 4 usuarios hagan withdraw, también podemos ver que es mucho mas probable que 2 usuario hagan repay a que 7 usuarios lo hagan. Esta será la idea que implementaremos para modelar la ocurrencia de cada acción.

In [ ]:
def temporalBlock(market,blockNumber):  # Recibe el bloque anterior
    """
    Se crea una copia temporal del bloque sin acciones del mercado
    ...
    Attributes
    ----------
    market : list
        An list of all configurations of the market in blocks
    blockNumber : int 
        Number of the previous block
    """
    temporal = []
    for value in market[blockNumber][:10]:
        temporal.append(value)
    for user in market[blockNumber][10:]:
        temporal.append(("x",0,user[2],user[3],user[4],user[5]))
    return temporal

def userAtBlock(markets,blockNumber,userIndex):
    """
    Devuelve el estado en cada mercado en bloque especifico de un usuario fijo
    ...
    Attributes
    ----------
    markets : list
        An list of all configurations of the markets in blocks
    blockNumber : int 
        Number of the previous block
    userIndex : int
        Index of the account
    ...
    returns
    -------
    list with the latest account report in each market    
    """
    temporalUserAtBlock = []
    for market in markets:
        temporalUserAtBlock.append(market[blockNumber][userIndex])
    return temporalUserAtBlock

def accountLiquidity(markets,blockNumber,collateralFactors,userIndex): # Recibe el ultimo bloque creado
    """Nos indica la liquidez y el deficit de la cuenta"""
    tmpUserAtBlock = userAtBlock(markets,blockNumber,userIndex)
    collateralBalance = 0  # Valor en USD
    borrowBalance = 0  # Valor en USD
    for m in range(len(tmpUserAtBlock)):
        index = markets[m][blockNumber][9][-1]/markets[m][blockNumber][9][tmpUserAtBlock[m][5]]
        collateralBalance += (tmpUserAtBlock[m][3]/markets[m][blockNumber][4])*collateralFactors[m]*markets[m][blockNumber][2]
        borrowBalance += (tmpUserAtBlock[m][4]*index)*markets[m][blockNumber][2]
    shortfall = max(borrowBalance - collateralBalance,0)  # Cuanto se deberia liquidar
    liquidity = max(collateralBalance - borrowBalance,0)  # Cuanto le queda de colchon para pedir prestamos
    return (liquidity,shortfall)   #valores en USD
    
accountLiquidity([M1,M2],0,[0.75,0.75],10)

A continuación definiremos cada uno de los posibles eventos que se pueden presentar en cada bloque:

1.   **Nuevos suppliers**



![texto del enlace](Imagenes/supply.jpg)

In [ ]:
def newSuppliers(market,estimatedOccurrences,blockNumber,maxSupply,restriction=True):  # Recibe el bloque anterior
    # Copia temporal del mercado con la copia temporal del bloque anterior
    # -----------------------------------------------------------------------------------------------------
    previousBlockCopy = temporalBlock(market,blockNumber)  # Esta es la que se edita
    # -----------------------------------------------------------------------------------------------------
    # Llegada aleatoria de suppliers
    # -----------------------------------------------------------------------------------------------------
    poissonDistribution = ss.poisson(estimatedOccurrences)  # Se aplica una distribución de Poisson
    n_users = len(previousBlockCopy[10:])
    numberOfPeople = np.arange(n_users)
    probability = [] 
    for i in numberOfPeople:
        probability.append(poissonDistribution.pmf(i))
    numberOfNewSuppliers = random.choices(numberOfPeople,probability)[0]  # Cantidad de nuevos suppliers
    # -----------------------------------------------------------------------------------------------------
    # Se guardan los indices de los usuarios
    # -----------------------------------------------------------------------------------------------------
    users = []
    userIndex = 10
    for user in previousBlockCopy[10:]:
        users.append((user,userIndex))
        userIndex += 1
    # -----------------------------------------------------------------------------------------------------
    # Se registran los nuevos suppliers
    # -----------------------------------------------------------------------------------------------------
    for newSupplier in range(numberOfNewSuppliers):
        user, userIndex = random.sample(users,1)[0]
        users.remove((user, userIndex))
        liquidity,shortfall = accountLiquidity(markets,blockNumber,collateralFactors,userIndex)
        if liquidity > 0:  # La cuenta no esta en zona de liquidación
            indexes = market[blockNumber][9]
            updatedSupply = user[3]/market[blockNumber][4]  # Tokens suplidos con intereses
            updatedBorrow = user[4]*(indexes[-1]/indexes[user[5]])  # Tokens en prestamo con intereses
            if restriction == True:  # Si no se permite el apalancamiento
                if updatedBorrow == 0:   
                    supplied = random.uniform(minAmount,maxSupply)
                    user = ("s",supplied,updatedSupply+supplied,(updatedSupply+supplied)*market[blockNumber][4],
                            updatedBorrow,len(indexes)-1)
                    previousBlockCopy[userIndex] = user
            else:  # Se permite el apalancamiento
                supplied = random.uniform(minAmount,maxSupply)
                user = ("s",supplied,updatedSupply+supplied,(updatedSupply+supplied)*market[blockNumber][4],
                        updatedBorrow,len(indexes)-1)
                previousBlockCopy[userIndex] = user
        else:
            previousBlockCopy[userIndex] = user
    # -----------------------------------------------------------------------------------------------------
    return previousBlockCopy

2.  **Nuevos borrowers**

![texto del enlace](Imagenes/borrow.jpg)

Para determinar la capacidad de prestamo de cada usuario, se deben tener en cuenta los movimientos del mismo usuario en todos los mercados, el siguiente diagrama nos permite determinar el valor de la capacidad de prestamo:

![texto del enlace](Imagenes/borrow_limit.jpg)

Las casillas que vemos en el paso intermedio del diagrama anterior, corresponde con los balances del usuario en cada mercado, al realizar el dichos calculos se suman los valores obtenidos y el resultado será la capacidad de préstamo del usuario.

In [ ]:
# marketBor --> Mercado donde se va a pedir prestado
def newBorrowers(markets,marketBor,estimatedOccurrences,blockNumber,collateralFactors):  # Recibe el bloque anterior
    # Copia temporal del mercado con la copia temporal del bloque anterior
    # -----------------------------------------------------------------------------------------------------
    previousBlockCopy = temporalBlock(marketBor,blockNumber)  # Esta es la que se edita
    # -----------------------------------------------------------------------------------------------------
    # Llegada aleatoria de borrowers
    # -----------------------------------------------------------------------------------------------------
    poissonDistribution = ss.poisson(estimatedOccurrences)  # Distribución de Possion
    n_users = len(previousBlockCopy[10:])
    numberOfPeople = np.arange(n_users)
    probability = []
    for i in numberOfPeople:
        probability.append(poissonDistribution.pmf(i))
    numberOfNewBorrowers = random.choices(numberOfPeople,probability)[0]
    # -----------------------------------------------------------------------------------------------------
    # Se guardan los indices de los usuarios
    # -----------------------------------------------------------------------------------------------------
    users = []
    userIndex = 10
    for user in previousBlockCopy[10:]:
        users.append((user,userIndex))
        userIndex += 1
    # -----------------------------------------------------------------------------------------------------
    # Se registran los nuevos borrowers
    # -----------------------------------------------------------------------------------------------------
    for newBorrower in range(numberOfNewBorrowers):
        user, userIndex = random.sample(users,1)[0]
        users.remove((user, userIndex))
        liquidity,shortfall = accountLiquidity(markets,blockNumber,collateralFactors,userIndex)
        if liquidity > 0:  # Si la cuenta no esta en zona de liquidación
            indexes = marketBor[blockNumber][9]
            updatedSupply = user[3]/marketBor[blockNumber][4]
            updatedBorrow = user[4]*(indexes[-1]/indexes[user[5]])
            marketLiquidity = marketBor[blockNumber][8]  # Cash del mercado
            liquidityInTokens = liquidity/marketBor[blockNumber][2]
            limit = min(liquidityInTokens,marketLiquidity)  # Se revisa si hay suficiente cash en el mercado
            if limit > minAmount:
                borrowed = random.uniform(minAmount,limit)
                user = ("b",borrowed,updatedSupply,user[3],updatedBorrow+borrowed,len(indexes)-1)  
                previousBlockCopy[userIndex]=user                                           
        else:
            previousBlockCopy[userIndex]=user 
    # -----------------------------------------------------------------------------------------------------
    return previousBlockCopy

3.   **Nuevos withdraws**

![texto del enlace](Imagenes/withdraw.jpg)

En el diagráma anterior aparecen dos nuevos conceptos que no habiamos mencionado, el **ratio** del usuario y la **liquidez del mercado**.

Recordemos que el collateral factor nos indicaba el porcentaje de colateral que podiamos tomar prestado, el ratio es el reciproco del collateral factor, es decir, el ratio nos permite determinar cual es la cantidad de collateral que necesitamos para dar garantia sobre el prestamo que tenemos.

Por ejemplo, supongamos que tenemos un deposito de 100 tokens en el mercado $MK1$, el cual cuenta con un collateral factor de $75\%$, esto nos indica que podemos realizar un prestamo de hasta 75 tokens en el mercado. Supongamos que realizamos dicho prestamo y que posteriormente pagamos 50 tokens de la deuda, quedandonos asi un saldo de 25 tokens en el prestamo (para el ejemplo no tomaremos en cuenta los intereses), la pregunta sería que tanto podemos sacar del colateral que teniamos? La respuesta nos la da el ratio y se calcula de la siguiente manera

\begin{align}
ratio &= \frac{1}{collateralFactor}*borrowBalance \\
&=\frac{1}{75\%}*25\\
&=33.3
\end{align}

De ese modo podremos realizar un withdraw de hasta $66.6\text{ tokens} = 100-33.3$.

Por otro lado tenemos la liquidez del mercado, la cual en terminos concretos es la cantidad de tokens que hay en el mercado, incluyendo las reservas. Cabe resaltar que si se retiran tokens de la reserva se esta adquiriendo riesgo de liquidez en el mercado. 

La manera en la que se calcula la liquidez del mercado es 

$$\text{liquidity}=\text{total supply}-\text{total borrow balance}$$



In [ ]:
# Nuevos Withdraws
def newWithdraws(markets,marketWith,estimatedOccurrences,blockNumber,collateralFactors): # Recibe el bloque anterior
    # Copia temporal del mercado con la copia temporal del bloque anterior
    # -----------------------------------------------------------------------------------------------------
    previousBlockCopy = temporalBlock(marketWith,blockNumber)  # Esta es la que se edita
    # -----------------------------------------------------------------------------------------------------
    # Se guardan los indices de los usuarios
    # -----------------------------------------------------------------------------------------------------
    suppliers = []
    users = []
    userIndex = 10
    for user in previousBlockCopy[10:]:
        if user[3] > 0:
            suppliers.append((user,userIndex))
        users.append((user,userIndex))
        userIndex += 1
    numberOfPeople = np.arange(len(suppliers))
    # -----------------------------------------------------------------------------------------------------
    # Llegada aleatoria de withdraws
    # -----------------------------------------------------------------------------------------------------
    poissonDistribution = ss.poisson(estimatedOccurrences)  # Distribución de Possion
    probability = []
    for i in numberOfPeople:
        probability.append(poissonDistribution.pmf(i))
    if len(numberOfPeople) == 0:
        return previousBlockCopy
    numberOfNewWithdraws = random.choices(numberOfPeople,probability)[0]
    # -----------------------------------------------------------------------------------------------------
    # Se registran los nuevos withdraws
    # -----------------------------------------------------------------------------------------------------
    numberOfNewWithdraws = min(numberOfNewWithdraws, len(suppliers))
    for newWith in range(numberOfNewWithdraws):
        user, userIndex = random.sample(users,1)[0]
        users.remove((user, userIndex))
        liquidity,shortfall = accountLiquidity(markets,blockNumber,collateralFactors,userIndex)
        if liquidity > 0:
            indexes = marketWith[blockNumber][9]
            updatedSupply = user[3]/marketWith[blockNumber][4]
            updatedBorrow = user[4]*(indexes[-1]/indexes[user[5]])
            liquidityInTokens = liquidity/marketWith[blockNumber][2]
            marketLiquidity = marketWith[blockNumber][8] + marketWith[blockNumber][3] # Cash + Reservas del mercado
            limit = min(updatedSupply,liquidityInTokens,marketLiquidity)
            if limit > minAmount:
                withdrawn = random.uniform(minAmount,limit)
                user = ("w",withdrawn,updatedSupply-withdrawn,(updatedSupply-withdrawn)*marketWith[blockNumber][4],
                        updatedBorrow, len(indexes)-1)
                previousBlockCopy[userIndex]=user
        else:
            previousBlockCopy[userIndex]=user
    return previousBlockCopy

printMarket(M1)
newWith = newWithdraws([M1,M2],M1,2,0,[0.2,0.2])
printMarket([newWith])

4.  **Nuevos repays**

![texto del enlace](Imagenes/repay.jpg)

In [ ]:
# Nuevos Repays
def newRepays(market, estimatedOccurrences, blockNumber):  # Recibe el bloque anterior
    # Copia temporal del mercado con la copia temporal del bloque anterior
    # -----------------------------------------------------------------------------------------------------
    previousBlockCopy = temporalBlock(market,blockNumber)  # Esta es la que se edita
    # -----------------------------------------------------------------------------------------------------
    # Se guardan los indices de los usuarios
    # -----------------------------------------------------------------------------------------------------
    borrowers = []
    users = []
    userIndex = 10
    for user in previousBlockCopy[10:]:
        if user[4] > 0:
            borrowers.append((user,userIndex))
        users.append((user,userIndex))
        userIndex += 1
    numberOfPeople = np.arange(len(borrowers))
    if len(numberOfPeople) == 0:
        return previousBlockCopy
    # -----------------------------------------------------------------------------------------------------
    # Llegada aleatoria de withdraws
    # -----------------------------------------------------------------------------------------------------
    poissonDistribution = ss.poisson(estimatedOccurrences)  # Distribución de Possion
    probability = []
    for i in numberOfPeople:
        probability.append(poissonDistribution.pmf(i))
    numberOfNewRepays = random.choices(numberOfPeople,probability)[0]
    numberOfNewRepays = min(numberOfNewRepays,len(borrowers))
    # -----------------------------------------------------------------------------------------------------
    # Se registran los nuevos withdraws
    # -----------------------------------------------------------------------------------------------------
    for newRepay in range(numberOfNewRepays):
        user, userIndex = random.sample(users,1)[0]
        users.remove((user, userIndex))
        liquidity,shortfall = accountLiquidity(markets,blockNumber,collateralFactors,userIndex)
        if liquidity > 0:
            indexes = market[blockNumber][9]
            updatedSupply = user[3]/market[blockNumber][4]
            updatedBorrow = user[4]*(indexes[-1]/indexes[user[5]])
            if updatedBorrow > 0:
                repay = random.uniform(minAmount,updatedBorrow) 
                user = ("r",repay, updatedSupply, user[3], updatedBorrow-repay, len(indexes)-1)
            previousBlockCopy[userIndex]=user
        else:
            previousBlockCopy[userIndex]=user
    return previousBlockCopy

In [ ]:
# Valores para los lambda
suppliersPerBlock = 3 # suppliersPerBlock
borrowersPerBlock = 3 # borrowersPerBlock
withdrwasPerBlock = 3 # withdrwasPerBlock
repaysPerBlock = 3 # repaysPerBlock

def newBlockMarket(markets,market,collateralFactors,blockNumber,reserveFactor,priceMarket,interestModel,maxSupply,restriction=False):
    # Copia temporal del mercado con la copia temporal del bloque anterior
    # -----------------------------------------------------------------------------------------------------
    previousBlockCopy = temporalBlock(market,blockNumber-1)  # Esta es la que se edita
    # -----------------------------------------------------------------------------------------------------
    # Se actualizan y se guardan las variables del mercado en el nuevo bloque en base al bloque anterior
    # -----------------------------------------------------------------------------------------------------
    newBlockMarket = setMarketVariablesInBlock(market,blockNumber,reserveFactor,priceMarket,interestModel)  
    market.append(newBlockMarket)  # Se guarda el nuevo bloque en el mercado
    # -----------------------------------------------------------------------------------------------------
    # Se generan las posibles nuevas acciones de los usuarios
    # -----------------------------------------------------------------------------------------------------
    newSup = newSuppliers(market,suppliersPerBlock,blockNumber-1,maxSupply,restriction)  
    newBor = newBorrowers(markets,market,borrowersPerBlock,blockNumber-1,collateralFactors)
    newWith = newWithdraws(markets,market,withdrwasPerBlock,blockNumber-1,collateralFactors)
    newRep = newRepays(market,repaysPerBlock,blockNumber-1)  
    # -----------------------------------------------------------------------------------------------------
    # Se reportan las nuevas acciones
    # -----------------------------------------------------------------------------------------------------
    userIndex = 10
    for user in previousBlockCopy[10:]:
        actions = [newSup[userIndex],newBor[userIndex],newWith[userIndex],newRep[userIndex]]
        action = random.sample(actions,1)[0] 
        market[blockNumber][userIndex] = action
        userIndex += 1
    # -----------------------------------------------------------------------------------------------------
    return market

M1 = marketSetUp(5,0.2,0.5,10,50,getBorrowAPR_volatileMarket)
M2 = marketSetUp(5,0.15,0.8,5,20,getBorrowAPR_volatileMarket)
markets = [M1,M2]

MK1 = newBlockMarket(markets,M1,[0.75,0.75],1,0.2,10,getBorrowAPR_volatileMarket,50)
MK2 = newBlockMarket(markets,M2,[0.75,0.75],1,0.2,5,getBorrowAPR_volatileMarket,20)

print("======================================================= M1 ======================================================")
printMarket(M1)
print("\n======================================================= M2 =======================================================")
printMarket(M2)

In [ ]:
M1 = marketSetUp(5,0.2,0.5,10,50,getBorrowAPR_volatileMarket)
M2 = marketSetUp(5,0.15,0.8,5,20,getBorrowAPR_volatileMarket)
markets = [M1,M2]

for i in range(10):
    M1 = newBlockMarket(markets,M1,[0.75,0.75],i+1,0.2,10,getBorrowAPR_volatileMarket,50)
    M2 = newBlockMarket(markets,M2,[0.75,0.75],i+1,0.2,5,getBorrowAPR_volatileMarket,20)
    markets = [M1,M2]

In [ ]:
printMarket(M1)

In [ ]:
printMarket(M2)

## Generación del nuevo bloque

In [ ]:
liquidationIncentive = 0.08
closeFactor = 0.05

def totalBorrowUser(markets,blockNumber,userConfigurations,userIndex): # Recibe el nuevo bloque
    """
    Devuelve la deuda total del usuario en USD
    ...
    Attributes
    ----------
    markets : list
        An list of all configurations of the markets in blocks
    blockNumber : int 
        Number of the las generated block
    userConfigurations : list
        List with configurations of the user at each market
    userIndex : int
        Index of the account
    """
    borrow = 0
    AtMarkets = userAtBlock(markets,blockNumber,userIndex)
    for m in range(len(markets)):
        index = markets[m][blockNumber][9][-1]/markets[m][blockNumber][9][userConfigurations[m][5]]
        borrow += userConfigurations[m][4]*index*markets[m][blockNumber][2]
    return borrow  # En USD

userConfigurations = userAtBlock([M1,M2],1,10)
totalBorrowUser([M1,M2],1,userConfigurations,10)

In [ ]:
def payLoanPerMarket(pay,market,userConfigurationAtMarket,blockNumber,userIndex):
    """
    Devuelve una lista con lo que le queda de saldo al liquidador y la actualización de la 
    deuda del borrower en el mercado
    ...
    Attributes
    ----------
    pay : float
        Amount paid in debt by the liquidator
    market : list
        Market where the liquidator pays the debt
    blockNumber : int
        last block generated
    userConfigurations : list
        List with configurations of the user at each market
    userIndex : int
        Index of the account
    """
    index = market[blockNumber][9][-1]/market[blockNumber][9][userConfigurationAtMarket[5]]
    updatedBorrow = userConfigurationAtMarket[4]*index
    amount = pay/market[blockNumber][2]   # Valor del pago en tokens del mercado
    balance = updatedBorrow - amount
    updatedSupply = userConfigurationAtMarket[3]/market[blockNumber][4]
    if balance >= 0:
        user = ("x",0,updatedSupply,userConfigurationAtMarket[3],balance,len(market[blockNumber][9])-1)
        pay = 0  # El liquidador se gasto toda la plata
    else:
        user = ("x",0,updatedSupply,userConfigurationAtMarket[3],0,len(market[blockNumber][9])-1)
        pay = abs(balance)*market[blockNumber][2]   # Valor actualizado del pago en USD
    userConfigurationAtMarket = user
    return [pay, userConfigurationAtMarket]

In [ ]:
def payPerLiquidation(payPerLiq,market,blockNumber,userIndex):
    """Devuelve una lista con lo que queda por pagar al liquidador y la actualización del 
    colateral del borrower en el mercado
    ...
    Attributes
    ----------
    payPerLiq : float
        amount to be paid to the liquidator
    market : list
        Market in which the liquidator receives the collateral
    blockNumber : int
        last block generated
    userConfigurations : list
        List with configurations of the user at each market
    userIndex : int
        Index of the account
    """
    userConfigurationAtMarket = tuple(market[blockNumber][userIndex])
    index = market[blockNumber][9][-1]/market[blockNumber][9][userConfigurationAtMarket[5]]
    updatedBorrow = userConfigurationAtMarket[4]*index
    amount = payPerLiq/market[blockNumber][2]   # Valor del pago en tokens del mercado
    updatedSupply = userConfigurationAtMarket[3]/market[blockNumber][4]
    balance = updatedSupply - amount
    if balance >= 0:
        user = ("x",0,balance,balance*market[blockNumber][4],updatedBorrow,len(market[blockNumber][9])-1)
        payPerLiq = 0  # Se le pago todo al liquidador
    else:
        user = ("x",0,0,0,updatedBorrow,len(market[blockNumber][9])-1)
        payPerLiq = abs(balance)*market[blockNumber][2]   # Valor actualizado del pago en USD
    userConfigurationAtMarket = user
    return [payPerLiq, userConfigurationAtMarket]

payPerLiquidation(10.8,M1,1,10)

In [ ]:
def newLiquidator(markets,blockNumber,userConfigurationAtMarkets,pay,userIndex,liquidationIncentive=liquidationIncentive):
    """
    Devuelve las configuraciones del usuario en cada mercado luego de la liquidación
    """
    # ------------------------------------------------------------------------------------------------------------------
    # Valores en USD
    # ------------------------------------------------------------------------------------------------------------------
    totalPay = pay  # Valor que paga el liquidador en USD
    payPerLiq = totalPay*(1 + liquidationIncentive)  # Valor que debe recibir el liquidador en USD
    # ------------------------------------------------------------------------------------------------------------------
    # Mercados de mas volatil a mas estable
    # ------------------------------------------------------------------------------------------------------------------
    for m in range(len(userConfigurationAtMarkets)):
        updatedPayPerLiq = payPerLiquidation(payPerLiq,markets[m],blockNumber,userIndex)
        payPerLiq = updatedPayPerLiq[0]  # Monto que queda por pagar al liquidador en USD
        userConfigurationAtMarkets[m] = updatedPayPerLiq[1]  # Configuraciones luego de realizar los pagos al liquidador
        if payPerLiq == 0:   # Si ya no se le debe mas al liquidador, rompa el ciclo
            break
    # ------------------------------------------------------------------------------------------------------------------
    # Mercados de mas estable a mas volatil
    # ------------------------------------------------------------------------------------------------------------------
    userConfigurationAtMarkets = userConfigurationAtMarkets[::-1]
    markets = markets[::-1]
    for m in range(len(userConfigurationAtMarkets)):
        updatedPayLoan = payLoanPerMarket(pay,markets[m],userConfigurationAtMarkets[m],blockNumber,userIndex)
        pay = updatedPayLoan[0]  # Monto que le queda al liquidador por pagar
        userConfigurationAtMarkets[m] = updatedPayLoan[1]  # Configuraciones luego de realizar los pagos del liquidador
        if pay == 0:  # Si el liquidador se gasto toda la plata, rompa el ciclo
            break
    # ------------------------------------------------------------------------------------------------------------------
    # Se reordenan los mercados de mas volatil a mas estable
    # ------------------------------------------------------------------------------------------------------------------
    userConfigurationAtMarkets = userConfigurationAtMarkets[::-1]
    markets = markets[::-1]
    return userConfigurationAtMarkets

newLiquidator([M1,M2],1,userAtBlock([M1,M2],1,10),10,10)

In [ ]:
printMarket([M1[1]])
printMarket([M2[1]])

In [ ]:
def liquidation(markets,blockNumber,userIndex,collateralFactors,liquidationIncentive=liquidationIncentive,closeFactor=closeFactor,txFee=0,traddingFee=0):
    userConfigurationAtMarkets = userAtBlock(markets,blockNumber,userIndex)
    liquidity, shortfall = accountLiquidity(markets,blockNumber,collateralFactors,userIndex)
    totalPay = 0
    if shortfall > 0:
        borrow = totalBorrowUser(markets,blockNumber,userConfigurationAtMarkets,userIndex)  # Deuda total de la cuenta
        totalBorrow = borrow
        pay = totalBorrow*closeFactor  # ¡¡¡Este valor debe cambiarse de manera aleatoria para la siguiente versión!!!
        profit = pay*(1+liquidationIncentive) - pay - txFee - traddingFee  # Ganancias del liquidador
        while shortfall > 0 and borrow != 0 and profit > 0:
            totalPay += pay  # Se va guardando la información de los pagos
            configurationsAfterLiquidation = newLiquidator(markets,blockNumber,userConfigurationAtMarkets,pay,userIndex,
                                                           liquidationIncentive)
            for m in range(len(userConfigurationAtMarkets)): # Se guardan los efectos de la liquidación
                userConfigurationAtMarkets[m] = configurationsAfterLiquidation[m]
            # Se recalculan los valores a ver si entran mas liquidadores
            liquidity, shortfall = accountLiquidity(markets,blockNumber,collateralFactors,userIndex)
            borrow = totalBorrowUser(markets,blockNumber,userConfigurationAtMarkets,userIndex)
            profit = pay*(1+liquidationIncentive) - pay - txFee - traddingFee
        for m in range(len(userConfigurationAtMarkets)):
            userConfigurationAtMarket = ("l",totalPay,userConfigurationAtMarkets[m][2],userConfigurationAtMarkets[m][3],
                                         userConfigurationAtMarkets[m][4],userConfigurationAtMarkets[m][5])
            userConfigurationAtMarkets[m] = userConfigurationAtMarket  
            # Se guarda la información de la liquidación en las configuraciones del mercado
        return userConfigurationAtMarkets
    else:
        return userConfigurationAtMarkets

# userliq = ("x",0,10,500,4,6)
# marketLiq = [[0.25,0.02,10,5,50,1,25,10,userliq,("x",0,10,500,2,4)]]
# market2 =[[0.2,0.1,5,15,45,0,0,50,("x",0,5,225,10,13),("x",0,10,450,2,4)]]
# markets = [marketLiq,market2]
# positionsUser = borrowPositions(markets,0,8)
# print(positionsUser)
# print(newLiquidator(markets,0,22.06492262371665,8))
# print(liquidation(markets,0,9,[0.75,0.75]))
# liquidation(markets,0,10,[0.75,0.75])
# positions = borrowPositions(markets,1,8)
# print(positions)
# liquidationIndicator(markets,0,positionsUser,[0.75,0.75],8)
# newLiquidator(markets,0,positionsUser,50,8)

# #printMarket([liquidation([M1,M2],1,10,[0.75,0.75])[0]])
liquidation([M1,M2],1,10,[0.75,0.75])

In [ ]:
printMarket([M1[1]])
printMarket([M2[1]])

In [ ]:
M1 = marketSetUp(5,0.2,0.5,10,50,getBorrowAPR_volatileMarket)
M2 = marketSetUp(5,0.15,0.8,5,20,getBorrowAPR_volatileMarket)
markets = [M1,M2]

printMarket(M1)
printMarket(M2)

In [ ]:
def newBlockWithLiquidations(markets,blockNumber,collateralFactors,reserveFactors,priceMarkets,interestModels,maxSupplies):
    for m in range(len(markets)):
        markets[m] = newBlockMarket(markets,markets[m],collateralFactors,blockNumber,reserveFactors[m],priceMarkets[m],
                                    interestModels[m],maxSupplies[m])
    userIndex = 10
    configurationsOfAllUsers = []
    for user in range(len(markets[0][blockNumber][10:])):
        configurations = liquidation(markets,blockNumber,userIndex,collateralFactors)
        configurationsOfAllUsers.append((userIndex,configurations))
        userIndex += 1
    for configuration in configurationsOfAllUsers:
        userIndex, userConfigurations = configuration
        for m in range(len(userConfigurations)):
            markets[m][blockNumber][userIndex] = userConfigurations[m]
    return markets

M1 = marketSetUp(5,0.2,0.5,10,50,getBorrowAPR_volatileMarket)
M2 = marketSetUp(5,0.15,0.8,1.01,20,getBorrowAPR_stableMartet)
markets = [M1,M2]

collateralFactors = [0.75,0.75]
priceMarkets = [10,1.01]
interestModels = [getBorrowAPR_volatileMarket,getBorrowAPR_stableMartet]
maxSupplies = [50,20]

newMarkets = newBlockWithLiquidations(markets,1,collateralFactors,reserveFactors,priceMarkets,interestModels,maxSupplies)
newMarkets = newBlockWithLiquidations(newMarkets,2,collateralFactors,reserveFactors,priceMarkets,interestModels,maxSupplies)
newMarkets = newBlockWithLiquidations(newMarkets,3,collateralFactors,reserveFactors,priceMarkets,interestModels,maxSupplies)
newMarkets = newBlockWithLiquidations(newMarkets,4,collateralFactors,reserveFactors,priceMarkets,interestModels,maxSupplies)
# newBlock
printMarket(newMarkets[0])
printMarket(newMarkets[1])

In [ ]:
M1 = marketSetUp(5,0.2,0.5,10,50,getBorrowAPR_volatileMarket)
M2 = marketSetUp(5,0.15,0.8,1.01,20,getBorrowAPR_stableMartet)
markets = [M1,M2]

printMarket(M1)
printMarket(M2)

In [ ]:
collateralFactors = [0.75,0.75]
priceMarkets = [10,1.01]
interestModels = [getBorrowAPR_volatileMarket,getBorrowAPR_stableMartet]
maxSupplies = [50,20]
newBlock = markets

for i in range(1,101):
    newBlock = newBlockWithLiquidations(newBlock,i,collateralFactors,reserveFactors,priceMarkets,interestModels,maxSupplies)
# newBlock

In [ ]:
printMarket(newBlock[0][15:17])

In [ ]:
printMarket(newBlock[1])

In [ ]:
def preutilization_volatileMarket(borrowRate,baseBorrowRate=0.02,slope=0.1):
    return (borrowRate - baseBorrowRate)/slope

def preutilization_stableMarket(borrowRate,baseBorrowRate=0.02,slope1=0.05,slope2=1,optimalUtilization=0.8):
    if borrowRate <= getBorrowAPR_stableMartet(optimalUtilization):
        return (borrowRate - baseBorrowRate)/slope1
    else:
        return (borrowRate - (baseBorrowRate+(slope1-slope2)*optimalUtilization))/slope2

In [ ]:
Er1 = []; Er2 = []  # ExchangeRate
Br1 = []; Br2 = []  # BorrowRate  
Ut1 = []; Ut2 = []  # UtilizationRate 
Sr1 = []; Sr2 = []  # SupplyRate
TB1 = []; TB2 = []  # TotalBorrow
Cs1 = []; Cs2 = []  # Cash
TR1 = []; TR2 = []  # TotalReserves
TS1 = []; TS2 = []  # TotalSupply

for block in newBlock[0]:
    Er1.append(block[4]); Er2.append(block[1][4])
    Br1.append(block[0]); Br2.append(block[1][0])
    Sr1.append(block[1]); Sr2.append(block[1][1])
    Ut1.append(preutilization_volatileMarket(block[0][0])); Ut2.append(preutilization_stableMarket(block[1][0]))
    TB1.append(block[0][6]); TB2.append(block[1][6])
    Cs1.append(block[0][8]); Cs2.append(block[1][8])
    TR1.append(block[0][3]); TR2.append(block[1][3])
    TS1.append(block[0][7]); TS2.append(block[1][7])

In [ ]:
plt.plot(Sr1)

In [ ]:
figsize(16,10)

fig, axs = plt.subplots(3, 3)
# UtilizationRate
axs[0, 0].plot(Ut1); axs[0, 0].plot(Ut2)
axs[0, 0].set(xlabel="Blocks", ylabel="Rate"); axs[0, 0].set_title('UtilizationRate')
# BorrowRate
axs[0, 1].plot(Br1); axs[0, 1].plot(Br2) 
axs[0, 1].set(xlabel="Blocks", ylabel="Rate"); axs[0, 1].set_title('BorrowRate')
# SupplyRate
axs[0, 2].plot(Sr1); axs[0, 2].plot(Sr2) 
axs[0, 2].set(xlabel="Blocks", ylabel="Rate"); axs[0, 2].set_title('SupplyRate')
# borrowRate vs utilizationRate
axs[1, 0].scatter(Ut1,Br1); axs[1, 0].scatter(Ut2,Br2)
axs[1, 0].set(xlabel="UtilizationRate", ylabel="borrowRate"); axs[1, 0].set_title('borrowRate vs utilizationRate')
# SupplyRate vs utilizationRate
axs[1, 1].scatter(Ut1,Sr1); axs[1, 1].scatter(Ut2,Sr2)
axs[1, 1].set(xlabel="UtilizationRate", ylabel="SupplyRate"); axs[1, 1].set_title('SupplyRate vs utilizationRate')
# TotalBorrow
axs[1, 2].plot(TB1); axs[1, 2].plot(TB2)
axs[1, 2].set(xlabel="Blocks", ylabel="TotalBorrow"); axs[1, 2].set_title('TotalBorrow')
# TotalReserves
axs[2, 0].plot(TR1); axs[2, 0].plot(TR2)
axs[2, 0].set(xlabel="Blocks", ylabel="TotalReserves"); axs[2, 0].set_title('TotalReserves')
fig.tight_layout()

In [ ]:
# TotalReserves
plt.
plt.
plt.title("TotalReserves")
plt.xlabel("Blocks")
plt.ylabel("")
plt.show()

In [ ]:
# TotalSupply
plt.plot(TS1)
plt.plot(TS2)
plt.title("TotalSupply")
plt.xlabel("Blocks")
plt.ylabel("TotalSupply")
plt.show()

In [ ]:
# Cash
plt.plot(Cs1)
plt.plot(Cs2)
plt.title("Cash")
plt.xlabel("Blocks")
plt.ylabel("Cash")
plt.show()

In [ ]:
# ExchangeRate
plt.plot(ER1)
plt.plot(ER2)
bottom = 50-1000000*minAmount; top = 50+1000000*minAmount
plt.ylim((bottom,top))
plt.title("ExchangeRate")
plt.xlabel("Blocks")
plt.ylabel("ExchangeRate")
plt.show()

In [ ]:
def updateLiquidations(markets,blockNumber,collateralFactors,liquidationIncentive=liquidationIncentive,closeFactor=closeFactor):
    # Recibe el nuevo bloque
    

In [ ]:
TRU_supply = [6000,1000,5000,6000,1000,8000,1000,1500,2000,2000]
TRU_borrow = [2000,1500,8000,1000,500,5000,300,500,1000,2000]
TRU_withdraw = [1000,500,2000,450,1500,200,250,1000,2000,200]
TRU_repays = [150,900,500,800,200,500,250,1000,1000,1000]
suppliersPerDay = sum(TRU_supply)/len(TRU_supply)
suppliersPerBlock = suppliersPerDay/(4*60*24)
borrowersPerDay = sum(TRU_borrow)/len(TRU_borrow)
borrowersPerBlock = borrowersPerDay/(4*60*24)
withdrwasPerDay = sum(TRU_withdraw)/len(TRU_withdraw)
withdrwasPerBlock = withdrwasPerDay/(4*60*24)
repaysPerDay = sum(TRU_repays)/len(TRU_repays)
repaysPerBlock = repaysPerDay/(4*60*24)
#people = int(suppliersPerBlock+borrowersPerBlock+withdrwasPerBlock+repaysPerBlock)

print(suppliersPerBlock, borrowersPerBlock,withdrwasPerBlock,repaysPerBlock)
#print(suppliersPerBlock/people,borrowersPerBlock/people,withdrwasPerBlock/people,repaysPerBlock/people)

In [ ]:
def preferences(market,blockNumber):
    initialPriceMarket = market[0][2]

In [ ]:
MK1 = marketSetUp(5,0.2,0.5,10,50,getBorrowAPR_volatileMarket)
MK2 = marketSetUp(5,0.15,0.8,5,20,getBorrowAPR_volatileMarket)
markets = [MK1,MK2]

for i in range(10):
    newBlockMarket(markets,MK1,collateralFactors,i+1,0.2,11,getBorrowAPR_volatileMarket,50)
    newBlockMarket(markets,MK2,collateralFactors,i+1,0.15,11,getBorrowAPR_volatileMarket,20)
    markets = [MK1,MK2]

In [ ]:
printMarket(MK1)

In [ ]:
printMarket(MK2)

In [ ]:
def newStateMarkets(markets,collateralFactors,blockNumber,reserveFactors,priceMarkets,interestModels,minSupplyPerMarkets,maxSupplyPerMarkets,restriction=True,liquidationIncentive=liquidationIncentive,closeFactor=closeFactor):
    for m in range(len(markets)):
        markets[m] = newBlockMarket(markets,markets[m],collateralFactors,blockNumber,reserveFactors[m],priceMarkets[blockNumber][m],interestModels[m],minSupplyPerMarkets[m],maxSupplyPerMarkets[m],restriction)
    for userIndex in range(len(markets[0][blockNumber][8:])):
        liquidations = liquidation(markets,blockNumber,userIndex+8,collateralFactors,liquidationIncentive,closeFactor)
        for p in range(len(markets)):
            markets[p][blockNumber][userIndex+8] = liquidations[p]
    return markets

userliq = ("x",0,10,500,4,6)
marketLiq = [[0.25,0.02,10,5,50,1,25,10,userliq,("x",0,10,500,2,4)]]
market2 =[[0.2,0.1,5,15,45,0,0,50,("x",0,5,225,10,13),("x",0,10,450,2,4)]]
markets = [marketLiq,market2]
newState = newStateMarkets(markets,[0.75,0.75],1,[0.2,0.15],[[0,0],[10,5],[10,5]],[getBorrowAPR_volatileMarket,getBorrowAPR_volatileMarket],[0,0],[0,0])
markets = newState
newState = newStateMarkets(markets,[0.75,0.75],2,[0.2,0.15],[[0,0],[10,5],[10,5]],[getBorrowAPR_volatileMarket,getBorrowAPR_volatileMarket],[0,0],[0,0])
printMarket(newState[0])
print("---------------------------------------")
printMarket(newState[1])

In [ ]:
from IPython.core.pylabtools import figsize
figsize(16,6)

In [ ]:
n_users = 5
MK1 = marketSetUp(n_users,0.2,0.7,5,0,50,getBorrowAPR_MK1)
MK2 = marketSetUp(n_users,0.15,0.5,10,0,20,getBorrowAPR_MK2)
markets = [MK1,MK2]

n_iterations =  4*60*24
for i in range(n_iterations):
  MK1 = newBlockMarket(markets,MK1,collateralFactors,i+1,0.2,5,getBorrowAPR_MK1,0,50)
  MK2 = newBlockMarket(markets,MK2,collateralFactors,i+1,0.2,12,getBorrowAPR_MK2,0,20)
  if MK1[i+1][4]<0 or MK2[i+1][4]<0:
    print("Desde aquí se jodio la cosa", i)
    break
  markets = [MK1,MK2]

# plot supply rate
ur1 = []
ur2 = []
for i in range(len(MK1)):
  ur_MK1 = preutilization(MK1[i][0])
  ur_MK2 = preutilization(MK2[i][0])
  ur1.append(ur_MK1)
  ur2.append(ur_MK2)

# Borrow APR 
b1 = []
b2 = []
for i in range(len(MK1)):
  b_MK1 = MK1[i][0]
  b_MK2 = MK2[i][0]
  b1.append(b_MK1)
  b2.append(b_MK2)

#ur1 = sorted(ur1)
#ur2 = sorted(ur2)
#b1 = sorted(b1)
#b2 = sorted(b2)

plt.title("BorrowRate vs UtilizationRate")
plt.ylabel("BorrowRate")
plt.xlabel("UtilizationRate")
#plt.xlim((0,1))
#plt.ylim((0,0.2))
plt.scatter(ur1,b1)
plt.scatter(ur2,b2)

In [ ]:
printMarket(MK1)

In [ ]:
printMarket(MK2)

In [ ]:
n_iterations =  4*60*24
prices = []
for i in range(n_iterations):
  prices.append([10,5])

prices[:5]

In [ ]:
n_users = 10
MK1 = marketSetUp(n_users,0.2,0.7,5,0,50,getBorrowAPR_MK1)
MK2 = marketSetUp(n_users,0.15,0.5,10,0,20,getBorrowAPR_MK2)
markets = [MK1,MK2]
collateralFactors = [0.75,0.75]
reserveFactors = [0.2,0.15]
interestModels = [getBorrowAPR_MK1,getBorrowAPR_MK2]

n_iterations =  4*60*24
for i in range(n_iterations):
  markets = newStateMarkets(markets,collateralFactors,i+1,reserveFactors,prices,interestModels,[0,0],[50,50])
  if markets[0][i+1][4]<0 or markets[1][i+1][4]<0:
    print("Desde aquí se jodio la cosa", i)
    break

# plot supply rate
ur1 = []
ur2 = []
for i in range(len(markets[0])):
  ur_MK1 = getUtilizationRate(markets[0],i)
  ur_MK2 = getUtilizationRate(markets[1],i)
  ur1.append(ur_MK1)
  ur2.append(ur_MK2)

# Borrow APR 
b1 = []
b2 = []
for i in range(len(markets[0])):
  b_MK1 = markets[0][i][0]
  b_MK2 = markets[1][i][0]
  b1.append(b_MK1)
  b2.append(b_MK2)

ur1 = sorted(ur1)
ur2 = sorted(ur2)
b1 = sorted(b1)
b2 = sorted(b2)

plt.title("BorrowRate vs UtilizationRate")
plt.ylabel("BorrowRate")
plt.xlabel("UtilizationRate")
#plt.xlim((0,1))
#plt.ylim((0,0.2))
plt.scatter(ur1,b1)
plt.scatter(ur2,b2)

In [ ]:
printMarket(markets[0])
print("###############################################################################3")
printMarket(markets[1])

In [ ]:
# plot supply rate
t1 = []
t2 = []
for i in range(len(MK1)):
  t_MK1 = markets[0][i][5]
  t_MK2 = markets[0][i][5]
  t1.append(t_MK1)
  t2.append(t_MK2)

plt.title("Bloques sin acciones")
plt.ylabel("Cantidad de bloques")
plt.xlabel("bloques")
#plt.ylim(top=1.5)
plt.scatter(range(len(markets[0])),t1)
plt.scatter(range(len(markets[0])),t2)

In [ ]:
# Borrow APR 
s1 = []
s2 = []
for i in range(len(markets[0])):
  s_MK1 = markets[0][i][1]
  s_MK2 = markets[0][i][1]
  s1.append(s_MK1)
  s2.append(s_MK2)

s1 = sorted(s1)
s2 = sorted(s2)

plt.title("SupplyRate vs UtilizationRate")
plt.ylabel("SupplyRate")
plt.xlabel("UtilizationRate")
plt.xlim((0,1))
plt.ylim((0,0.1))
plt.scatter(ur1,s1)
plt.scatter(ur2,s2)

In [ ]:
reserve1 = []
reserve2 = []
k=0
for i in range(len(markets[0])):
  totalReserve1 = markets[0][i][3]
  totalReserve2 = markets[0][i][3]
  reserve1.append(totalReserve1)
  reserve2.append(totalReserve2)
  k += 1

plt.title("Reservas por bloque")
plt.ylabel("Precio de las reservas")
plt.xlabel("bloques")
plt.scatter(range(len(markets[0][:100])),reserve1[:100])
plt.scatter(range(len(markets[0][:100])),reserve2[:100])

In [ ]:
exc1 = []
exc2 = []
k=0
for i in range(len(markets[0])):
  exc1_1 = 1/markets[0][i][4]
  exc2_1 = 1/markets[0][i][4]
  exc1.append(exc1_1)
  exc2.append(exc2_1)
  k += 1

plt.title("Exchange por bloque")
plt.ylabel("Valor de los cTokens")
plt.xlabel("bloques")
plt.plot(range(len(markets[0][:100])),exc1[:100])
plt.plot(range(len(markets[0][:100])),exc2[:100])

In [ ]:
%matplotlib inline
from IPython.core.pylabtools import figsize
import pandas as pd

figsize(16,6)

x = np.linspace(0, 5)
N = ss.norm
USDCoin = N.pdf(x,1,0.1)  # mean = 15; sigma = 4
USDCoin_prices = []

n_iterations = 1000   #4*24*60
for iteration in range(n_iterations):
  price1 = random.choices(x,USDCoin)[0]
  USDCoin_prices.append(price1)

USDCoin = N.pdf(x,1,0.025)
n_iterations = 600   #4*24*60
for iteration in range(n_iterations):
  price1 = random.choices(x,USDCoin)[0]
  USDCoin_prices.append(price1)

#price_MK1 = []
#for i in range(n_iterations):
#  if i % 60 == 0:
#    price_MK1.append(MK1_prices[i])
#    price_MK2.append(MK2_prices[i])

plt.plot(range(1600),USDCoin_prices)
#plt.plot(range(int(n_iterations/60)),price_MK2)
#plt.ylim(top=60)

In [ ]:
Ether = pd.read_excel("Historico de precios/Ethereum.xlsx")
EtherPrices = Ether["PRECIO"][426:]
EtherPrices.plot()

In [ ]:
Bitcoin = pd.read_excel("Historico de precios/Bitcoin.xlsx")
BitcoinPrices = Bitcoin["PRECIO"][750:]
BitcoinPrices.plot()

In [ ]:
len(BitcoinPrices)

In [ ]:
# Collateral Factors - Uno por cada mercado

collFactor_ETH = 0.75  # Collateral factor para el mercado MK1
collFactor_BTC = 0.75  # Collateral factor para el mercado MK2   !!!!!Estimar!!!!
collFactor_USD = 0.75  # Collateral factor para el mercado MK3

# Lista con los Collateral Factors
collateralFactors = [collFactor_MK1,
                     collFactor_MK2]

# Reserve Factors - Uno por cada mercado

reserveFactor_ETH = 0.20   # Reserve Factor para el mercado MK1
reserveFactor_BTC = 0.15   # Reserve Factor para el mercado MK2    !!!Estimar!!!
reserveFactor_USD = 0.07   # Reserve Factor para el mercado MK3
 
# Lista con los Reserve Factors                    
reserveFactors = [reserveFactor_MK1, 
                  reserveFactor_MK2]

# Exchange rate iniciales

er_price_ETH = 50
er_price_BTC = 50
er_price_USD = 50

n_users = 10
initial_supply_ETH_Rate = 0.1
initial_supply_BTC_Rate = 0.25
initial_supply_USD_Rate = 0.05

ETH = marketSetUp(n_users,reserveFactor_ETH,initial_supply_ETH_Rate,er_price_ETH,Ether["PRECIO"][0])
BTC = marketSetUp(n_users,reserveFactor_BTC,initial_supply_BTC_Rate,er_price_BTC,Bitcoin["PRECIO"][0])
USD = marketSetUp(n_users,reserveFactor_USD,initial_supply_USD_Rate,er_price_USD,USDCoin_prices[0])
printMarket(ETH)
printMarket(BTC)
printMarket(USD)

In [ ]:
BinanceCoin = pd.read_excel("USDCoin.xlsx")
BinanceCoinPrices = BinanceCoin["PRECIO"]
BinanceCoinPrices.plot()

In [ ]:
n_users = 5

MK1 = marketSetUp(n_users,0.2,0.2,50,initialTokenPrices[0])
MK2 = marketSetUp(n_users,0.15,0.3,50,initialTokenPrices[1])
markets = [MK1,MK2]
n_iterations = 4*60*24
for i in range(n_iterations):
  MK1 = newBlockMarket(markets,MK1,collateralFactors,i+1,0.2,MK1_prices)
  MK2 = newBlockMarket(markets,MK2,collateralFactors,i+1,0.15,MK2_prices)
  markets = [MK1,MK2]

In [ ]:
# plot supply rate
ur1 = []
ur2 = []
for i in range(n_iterations):
  ur_MK1 = getUtilizationRate(MK1,i)
  ur_MK2 = getUtilizationRate(MK2,i)
  ur1.append(ur_MK1)
  ur2.append(ur_MK2)

plt.ylabel("utilization rate")
plt.xlabel("bloques")
plt.ylim(top=1.5)
plt.plot(range(n_iterations),ur1)
plt.plot(range(n_iterations),ur2)

In [ ]:
# plot supply rate
b1 = []
b2 = []
for i in range(n_iterations):
  b_MK1 = MK1[i][0]
  b_MK2 = MK2[i][0]
  b1.append(b_MK1)
  b2.append(b_MK2)

plt.ylabel("Borrow rate")
plt.xlabel("bloques")
plt.ylim(top=1.5)
plt.plot(range(n_iterations),b1)
plt.plot(range(n_iterations),b2)

In [ ]:
# plot supply rate
t1 = []
t2 = []
for i in range(n_iterations):
  t_MK1 = MK1[i][5]
  t_MK2 = MK2[i][5]
  t1.append(t_MK1)
  t2.append(t_MK2)

plt.title("Prestamos por bloque")
plt.ylabel("Total prestado")
plt.xlabel("bloques")
#plt.ylim(top=1.5)
plt.scatter(range(n_iterations),t1)
plt.scatter(range(n_iterations),t2)

In [ ]:
# plot borrow rate
variableHistory = []
borrow1 = []
borrow2 = []
util1 = []
util2 = []
k=0
for i in range(n_iterations):
  utilization1 = getUtilizationRate(MK1,k)
  utilization2 = getUtilizationRate(MK2,k) 
  util1.append(utilization1)
  util2.append(utilization2)
  borrow1.append(getBorrowAPR(utilization1))
  borrow2.append(getBorrowAPR(utilization2))
  k += 1

plt.title("BorrowRate vs UtilizationRate")
plt.ylabel("BorrowRate")
plt.xlabel("UtilizationRate")
plt.ylim(top=0.12)
plt.scatter(util1,borrow1)
plt.scatter(util2,borrow2)

In [ ]:
fig = go.Figure() #data=go.Scatter(x=x, y=y, name = "Borrow APR")

#fig.add_trace(go.Scatter(x=x,y=S,name="Supply APR"))
fig.add_trace(go.Scatter(x=util1,y=borrow1,name="Supply APR - simulación"))
fig.add_trace(go.Scatter(x=util2,y=borrow2,name="Borrow APR - simulación"))
fig.update_layout(xaxis_title='Tasa de utilización',
                  yaxis_title='Tasa APR')
fig.show()

In [ ]:
# plot supply rate
variableHistory = []
supply1 = []
supply2 = []
util1 = []
util2 = []
k=0
for i in range(n_iterations):
  utilization1 = getUtilizationRate(MK1,k)
  utilization2 = getUtilizationRate(MK2,k) 
  borrowRate1 = getBorrowAPR(utilization1)
  borrowRate2 = getBorrowAPR(utilization2)
  supplyRate1 = getSupplyAPR(borrowRate1,0.2,utilization1)
  supplyRate2 = getSupplyAPR(borrowRate2,0.15,utilization2)
  util1.append(utilization1)
  util2.append(utilization2)
  supply1.append(supplyRate1)
  supply2.append(supplyRate2)
  k += 1

plt.title("supplyRate vs UtilizationRate")
plt.ylabel("SupplyRate")
plt.xlabel("UtilizationRate")

plt.scatter(util1,supply1)
plt.scatter(util2,supply2)

In [ ]:
# precio de las reservas por bloque
reserve1 = []
reserve2 = []
k=0
for i in range(n_iterations):
  totalReserve1 = totalReserveMarket(MK1,k,0.2)*MK1_prices[k]
  totalReserve2 = totalReserveMarket(MK2,k,0.15)*MK2_prices[k]
  reserve1.append(totalReserve1)
  reserve2.append(totalReserve2)
  k += 1

plt.title("Reservas por bloque")
plt.ylabel("Precio de las reservas")
plt.xlabel("bloques")
plt.scatter(range(n_iterations),reserve1)
plt.scatter(range(n_iterations),reserve2)

In [ ]:
# precio de las reservas por bloque
reserve1 = []
reserve2 = []
k=0
for i in range(n_iterations):
  totalReserve1 = totalReserveMarket(MK1,k,0.2)
  totalReserve2 = totalReserveMarket(MK2,k,0.15)
  reserve1.append(totalReserve1)
  reserve2.append(totalReserve2)
  k += 1

plt.title("Reservas por bloque")
plt.ylabel("Precio de las reservas")
plt.xlabel("bloques")
plt.scatter(range(n_iterations),reserve1)
#plt.scatter(range(n_iterations),reserve2)

In [ ]:
plt.scatter(range(n_iterations),reserve2)

In [ ]:
# plot supply rate
totalBorrow1 = []
totalBorrow2 = []
for i in range(n_iterations):
  totalBorrow1.append(MK1[i][6])
  totalBorrow2.append(MK2[i][6])

plt.title("Prestamos por bloque")
plt.ylabel("Total prestado")
plt.xlabel("bloques")
#plt.scatter(range(n_iterations-10),totalBorrow1[10:])
plt.scatter(range(n_iterations-200),totalBorrow2[200:])

In [ ]:
# plot supply rate
totalBorrow1 = []
totalBorrow2 = []
for i in range(n_iterations):
  totalBorrow_MK1 = MK1[i][6]
  totalBorrow_MK2 = MK2[i][6]
  totalBorrow1.append(totalBorrow_MK1*MK1_prices[i])
  totalBorrow2.append(totalBorrow_MK2*MK2_prices[i])

plt.title("Prestamos por bloque")
plt.ylabel("Total prestado")
plt.xlabel("bloques")
plt.scatter(range(n_iterations-10),totalBorrow1[10:])
plt.scatter(range(n_iterations-10),totalBorrow2[10:])

In [ ]:
ctk1 = []
ctk2 = []
for i in range(n_iterations):
  ctk_MK1 = MK1[i][4]
  ctk_MK2 = MK2[i][4]
  ctk1.append(ctk_MK1)
  ctk2.append(ctk_MK2)

plt.title("Prestamos por bloque")
plt.ylabel("Total prestado")
plt.xlabel("bloques")
plt.scatter(range(n_iterations),ctk1)
plt.scatter(range(n_iterations),ctk2)

In [ ]:
MK1[0][2]

In [ ]:
def amountEstimation(market,blockNumber,mean=500,scale=0.5):
    if 0 <= market[blockNumber][0] < 0.125:
        collBorrow = mean*(1-3*scale)
        loanBorrow = mean*(1+3*scale)
        withdraw = mean*(1+(3/10)*scale)
    elif 0.125 <= market[blockNumber][0] < 0.25:
        collBorrow = mean*(1-2*scale)
        loanBorrow = mean*(1+2*scale)
        withdraw = mean*(1+(2/10)*scale)
    elif 0.25 <= market[blockNumber][0] < 0.5:
        collBorrow = mean*(1-scale)
        loanBorrow = mean*(1+scale)
        withdraw = mean*(1+(1/10)*scale)
    elif 0.5 <= market[blockNumber][0] < 0.75:
        collBorrow = mean*(1+scale)
        loanBorrow = mean*(1-scale)
        withdraw = mean*(1-(1/10)*scale)
    elif 0.75 <= market[blockNumber][0] < 0.875:
        collBorrow = mean*(1+2*scale)
        loanBorrow = mean*(1-2*scale)
        withdraw = mean*(1-(2/10)*scale)
    elif 0.875 <= market[blockNumber][0] <= 1:
        collBorrow = mean*(1+3*scale)
        loanBorrow = mean*(1-3*scale)
        withdraw = mean*(1-(3/10)*scale)
    amountPerAction = [collBorrow,loanBorrow,withdraw]
    for i in range(len(amountPerAction)):
        if amountPerAction[i] <= 0:
            amountPerAction[i] = 0 
    return amountPerAction

amountEstimation(MK1,0)